## Test API via UI

In [16]:
import requests
import json
import gradio as gr

from utils import returnSearchResults

In [17]:
def callSearchAPI(query : str):
    # Return top 5 search results for given query

    params = {"query" : query}
    response = requests.get("http://13.53.235.4:80/search", params=params)

    return json.loads(response.text)

In [18]:
def formatVideoEmbed(video_id : str):
    return '<iframe width="576" height="324" src="https://www.youtube.com/embed/'+ video_id +'"></iframe>'

def formatResultText(title : str, video_id : str):
    text = f"""<br> <br>
# {title}<br>

🔗 [Video Link](https://youtu.be/{video_id})"""
    
    return text

In [19]:
def searchResults(query : str):
    # Get top 5 search results
    response = callSearchAPI(query)

    # Format search results
    output_list = []   # Initialize output list

    num_empty_results = 5-len(response['title'])   # calculate number of null search results

    # display search results
    for i in range(len(response['title'])):
        video_id = response['video_id'][i]
        title = response['title'][i]

        embed = gr.HTML(value=formatVideoEmbed(video_id), visible=True)
        text = gr.Markdown(value=formatResultText(title, video_id), visible=True)

        output_list.append(embed)
        output_list.append(text)
    
    # Make null search result slots invisible
    for i in range(num_empty_results):

        # If no search results, then display 'No Results' text
        if num_empty_results==5 and i==0:
            embed = gr.HTML(visible=False)
            text = gr.Markdown(value="No results, perhaps try rephrasing your query.", visible = True)
        
            output_list.append(embed)
            output_list.append(text)
            continue
            
        embed = gr.HTML(visible=False)
        text = gr.Markdown(visible=False)

        output_list.append(embed)
        output_list.append(text)

    return output_list

In [20]:
# Create Gradio Demo
output_list = []

with gr.Blocks() as demo:
    gr.Markdown("# YouTube Search")

    with gr.Row():
        input_query = gr.Textbox(placeholder="What are you looking for?", label="Query", scale=3)
        button = gr.Button("Search")
        button.click(fn=searchResults, inputs=input_query, outputs=output_list)

    for i in range(5):
        with gr.Row():
            output_list.append(gr.HTML())
            output_list.append(gr.Markdown())

    input_query.submit(fn=searchResults, inputs=input_query, outputs=output_list)

demo.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
